In [86]:
import numpy as np
import csv, copy, itertools, math, random
from collections import Counter
from scipy.special import expit

import matplotlib.pyplot as plt
import pandas

In [85]:
def getDataType(data):
    for itr in range(len(data)):
        val_u = data[itr]
        if val_u == '?':
            pass
        else:
            return val_u.isalpha()

In [87]:
# filling in missing data - mode for stings and mean for numbers
def bestGuessData(data):
    if getDataType(data):  # if it's a string, find mode
        data_count_dict = Counter(data)
        return data_count_dict.most_common(1)[0][0]
    else:  # if numbers, find average
        data_wo_missing = [d for d in data if d != '?']
        data_wo_missing = [float(d) for d in data_wo_missing]
        return np.mean(data_wo_missing)

In [88]:
# takes the best guess and replaces the missing data
def replaceMissingData(filename):
    dat = csv.reader(open(filename, 'r'), delimiter=';')

    headers = ([h[0] for h in dat][0]).split(',')
    columns = []
    with open(filename, 'r') as f:
        reader = csv.DictReader(f)
        g = [r for r in reader]
        f.close()
        for j in range(len(headers)):
            for i in range(len(g)):
                columns.append([v for k, v in g[i].items() if k == headers[j]])
    f.close()
    columns = np.array(columns).reshape(len(headers), len(g))
    bg = []
    for i in range(len(headers)):
        bg.append(bestGuessData(columns[i][:]))

        columns[i][:] = [str(bg[i]) if x == '?' else x for x in columns[i][:]]
    dict_str_x = ['normal', 'yes', 'present', 'good', 'ckd']
    for i in range(len(columns)):
        if columns[i][0].isalpha():
            columns[i][:] = [1 if x in dict_str_x else 0 for x in columns[i][:]]

        else:
            pass
    columns = columns.astype(np.float64)

    return columns

In [131]:
def sigmoid(x):
    sig = 1.0 / (1.0 - ((np.exp(-x))))
#     print((expit(-x)==1))
    return sig

In [120]:
# hypohesis
def hyp_fn(w, x):
    z = np.dot(w, x)
    return sigmoid(z)

In [121]:
# comparing weights
def compareWights(old, curr):
    difference = curr - old
    return np.sqrt(np.sum(np.square(difference)))

In [122]:
# decrease weights usin gradient descent and return new weights
def gradientDescent(m, n, w, true_y, train_data, lambda_value, learning_rate, epsilon_tolerance):
    flag_converged = 0
    counter = 0
    w_old = np.copy(w)

    while True:
        #counter += 1
        hf = hyp_fn(w, train_data)
        temp = np.dot(hf - true_y, train_data[1:, :].T)
        w[0, 0] = w[0, 0] - (learning_rate / m) * np.sum(hf - true_y)  # +(lambda_value/m)*w[0,0]
        for itr in range(n - 1):
            w[:, 1:] = w[:, 1:] - (learning_rate / m) * np.sum(temp) + (lambda_value / m) * sum(w[:, 1:])
        if compareWights(w_old, w) < epsilon_tolerance:
            break

        else:
            w_old = np.copy(w)

    return w

In [123]:
# initial weights aer chosen to be "1.0" fo simplicity
def generateRandomInitialWeights(n):
    weights = []
    for i in range(n):
        # weights.append(random.uniform(0,1))
        weights.append(1.0)
    weights = np.reshape(np.array(weights), (1, n))
    return weights

In [124]:
# normalization of data so data is comparable
def normalizeData(d, n):
    for itr in range(n):
        mean = np.mean(d[itr])
        std = np.std(d[itr])
        d[itr] = (d[itr] - mean) / std
    return d

In [125]:
# finding weights needed to classify data
def findWeights(train_data, m, n, lambda_value, epsilon_tolerance, learning_rate):
    w_x = generateRandomInitialWeights(n)
    true_y = np.reshape(train_data[-1], (1, m))
    w_x = gradientDescent(m, n, w_x, true_y, train_data, lambda_value, learning_rate, epsilon_tolerance)
    return w_x

In [126]:
# computation of f measure, returns f measure
# compares true y with predicted y
def evaluatePridictedWithTrue(predicted_class, true_class):
    tp = np.sum([1.0 if (t == 1 and p == 1) else 0 for t, p in zip(predicted_class.T, true_class.T)])
    ##print "No. of true positives:", tp# for t,p in zip(predicted_class.T, true_class.T):
    fp = np.sum([1.0 if (p == 1 and t == 0) else 0 for t, p in zip(predicted_class.T, true_class.T)])
    # print "No. of false positives:", fp
    fn = np.sum([1.0 if (p == 0 and t == 1) else 0 for t, p in zip(predicted_class.T, true_class.T)])
    # print "No. of false negatives:", fn
    if (tp + fp) == 0 or (tp + fn) == 0:
        return np.nan
    else:
        pre = tp / (tp + fp)
        rec = tp / (tp + fn)
        return 2 * pre * rec / (pre + rec)

In [127]:
# predicted y s assessed using f measure
# and if the predicted y is > .5 the o/p is
# classified as y = 1, and 0 otherwise
def testing(x_test, y_test, wts_test):
    y_predicted = hyp_fn(wts_test, x_test)
    for i in range(80):
        op = y_predicted[0, i]
        if op > 0.5:
            y_predicted[0, i] = 1
        else:
            y_predicted[0, i] = 0
    # print "Truth value compared to the predicted class and the ground truth is :"
    f_meas = evaluatePridictedWithTrue(y_predicted, y_test)
    return f_meas

In [132]:
if __name__ == '__main__':
    # processing the data from files and relevant conversions
    file = 'csv_result-chronic_kidney_disease_full (1).csv'
    outfile = file
    data_column_wise = replaceMissingData(outfile)  # outfile must be csv
    n, no_samples = np.shape(data_column_wise)
    m1 = int(.1 * no_samples)
    m = int(.8 * no_samples)
    train_x = np.copy(data_column_wise[:n - 1, m1:m + m1])  # data for training - 80%
    train_y = np.copy(data_column_wise[n - 1:, m1:m + m1])
    lambda_range = [-2.0, -1.8, -1.6, -1.4, -1.2, -1.0, -.8, -.6, -.4, -.2, 0, .2, .4, .6, .8, 1.0, 1.2, 1.4, 1.6, 1.8,
                    2.0, 2.2, 2.4, 2.6, 2.8, 3.0, 3.2, 3.4, 3.6, 3.8, 4]
#     lambda_range=np.arange(-2.0,4.2,.2)
#     print(lambda_range)
    f_m = []
    for lambda_value in lambda_range:
        # print "lambda is =", lambda_value
        if lambda_value < -.6:
            alpha = .1049  # 0.1049#0.07
            epsilon_tolerance = 17000  # 17000# .5 for standardization
        elif lambda_value > -.6 and lambda_value < -.2:
            alpha = .001
            epsilon_tolerance = 315
        elif lambda_value >= -.2:  # and lambda_value<0 :
            alpha = .001
            epsilon_tolerance = 155

        wts = findWeights(train_x, m, n - 1, lambda_value, epsilon_tolerance, alpha)

        ################# Testing ######################

        m_test = no_samples - m
        test_x = np.copy(data_column_wise[:n - 1, :m1])  # data for testing - 20%
        test_x = np.concatenate((test_x, np.copy(data_column_wise[:n - 1, m1 + m:])), axis=1)
        y = np.copy(data_column_wise[n - 1:, :m1])  # true class
        y = np.concatenate((y, np.copy(data_column_wise[n - 1:, m1 + m:])), axis=1)
        temp_f_m = testing(test_x, y, wts)
        # print "f-meansure:", temp_f_m
        f_m.append(temp_f_m)
    # print 70*'_'
    # plotting f-measure vs lambda
    plt.figure("f-measure vs lambda")
    plt.title("f-measure vs lambda")
    plt.plot(lambda_range, f_m, 'ro')
    plt.xlabel("lambda = -2.0 to 4.0 in steps of 0.2")
    plt.ylabel("f-measure")
    plt.ylim((0, 1))
    plt.xlim((-2, 4))
    plt.show()

[[ 9612.46599658  8893.14332262  9777.41        8484.21
   9018.63987494  9038.83437334  6872.515       9070.70994752
  19324.405      22141.36599658 11516.43449953  8988.14487494
   4826.00751258  9124.47099658 10359.2722377   8921.02556352
  11960.751516    9027.92530957  7951.22        9016.39138545
   8980.06081004  9424.25081004  8075.42        8968.14487494
   8947.17854592  8924.13238752  8949.66081004  9219.91498262
   8931.19587152  7491.87099658  9036.515      15157.31
   8953.51688592  6861.10581084  6971.015       8851.23496144
   6805.305       7589.81        8999.11437447 12365.46599658
   9976.41        6223.28449953  8903.52530957  8871.85581004
   4387.81        8944.77202837  9200.78130615  6385.16099658
  13881.47693168  8871.36081004 10495.91       13085.215
   6241.01        7758.91       12393.61        8877.05581004
   8953.89587152  9558.015       6376.78449953 11158.51740793
   8885.95581004 11233.315       7422.91743494 13499.77099658
   8989.88130615  7311.81

ValueError: data type <class 'numpy.object_'> not inexact

In [43]:
###################################################################################################
###################################### Feature Normalization ######################################
###################################################################################################
import numpy as np
import csv, copy, itertools, math, random
from collections import Counter
import matplotlib.pyplot as plt

In [44]:
# this fn identifies the missing data positions by finding the data type
def getDataType(data):
    for itr in range(len(data)):
        val_u = data[itr]
        if val_u == '?':
            pass
        else:
            return val_u.isalpha()

In [45]:
# filling in missing data - mode for stings and mean for numbers
def bestGuessData(data):
    if getDataType(data):  # if it's a string, find mode
        data_count_dict = Counter(data)
        return data_count_dict.most_common(1)[0][0]
    else:  # if numbers, find average
        data_wo_missing = [d for d in data if d != '?']
        data_wo_missing = [float(d) for d in data_wo_missing]
        return np.mean(data_wo_missing)

In [46]:
# takes the best guess and replaces the missing data
def replaceMissingData(filename):
    dat = csv.reader(open(filename, 'r'), delimiter=';')

    headers = ([h[0] for h in dat][0]).split(',')
    columns = []
    with open(filename, 'r') as f:
        reader = csv.DictReader(f)
        g = [r for r in reader]
        f.close()
        for j in range(len(headers)):
            for i in range(len(g)):
                columns.append([v for k, v in g[i].items() if k == headers[j]])
    f.close()
    columns = np.array(columns).reshape(len(headers), len(g))
    bg = []
    for i in range(len(headers)):
        bg.append(bestGuessData(columns[i][:]))
        dummy_list = columns[i][:]
        columns[i][:] = [str(bg[i]) if x == '?' else x for x in columns[i][:]]
    dict_str_x = ['normal', 'yes', 'present', 'good', 'ckd']
    for i in range(len(columns)):
        if columns[i][0].isalpha():
            columns[i][:] = [1 if x in dict_str_x else 0 for x in columns[i][:]]

        else:
            pass
    columns = columns.astype(np.float)
    return columns

In [47]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [57]:
# hypohesis function for logit regression
def hyp_fn(w, x):
    z = np.dot(w, x)
    return sigmoid(z)

In [58]:
# compaing old and new weights for convergence criteria
def compareWights(old, curr):
    difference = (curr - old)
    difference = np.sqrt(np.sum(np.square(difference)))
    return difference

In [59]:
# decrease weights usin gradient descent and return new weights
def gradientDescent(m, n, w, true_y, train_data, lambda_value, learning_rate, epsilon_tolerance):
    flag_converged = 0
    counter = 0
    w_old = np.copy(w)
    # print "counting iterations...",
    while True:
        counter += 1
        hf = hyp_fn(w, train_data)
        temp = np.dot(hf - true_y, train_data[1:, :].T)
        w[0, 0] = w[0, 0] - (learning_rate / m) * np.sum(hf - true_y)  # +(lambda_value/m)*w[0,0]
        for itr in range(n - 1):
            w[:, 1:] = w[:, 1:] - (learning_rate / m) * np.sum(temp) + (lambda_value / m) * sum(w[:, 1:])
        if compareWights(w_old, w) < epsilon_tolerance:
            # print "\nWeights have converged",
            flag_converged = 1
            break
        else:
            w_old = np.copy(w)
            pass
    # print ".",
    # print "\n(converged in", counter-1, "iterations)"
    return w

In [60]:
# initial weights aer chosen to be "1.0" fo simplicity
def generateRandomInitialWeights(n):
    weights = []
    for i in range(n):
        weights.append(1.0)
    weights = np.reshape(np.array(weights), (1, n))
    return weights

In [61]:
# normalization of data so data is comparable
def normalizeData(d, n):
    for itr in range(n):
        mean = np.mean(d[itr])
        std = np.std(d[itr])
        d[itr] = (d[itr] - mean) / std
    return d

In [62]:
# finding weights needed to classify data
def findWeights(train_data, m, n, lambda_value, epsilon_tolerance, learning_rate):
    w_x = generateRandomInitialWeights(n)
    true_y = np.reshape(train_data[-1], (1, m))
    w_x = gradientDescent(m, n, w_x, true_y, train_data, lambda_value, learning_rate, epsilon_tolerance)
    return w_x

In [63]:
# computation of f measure, returns f measure
# compares true y with predicted ydef evaluatePridictedWithTrue(predicted_class, true_class):
def evaluatePridictedWithTrue(predicted_class, true_class):
    tp = np.sum([1.0 if (t == 1 and p == 1) else 0 for t, p in zip(predicted_class.T, true_class.T)])
    # print "No. of true positives:", tp# for t,p in zip(predicted_class.T, true_class.T):
    fp = np.sum([1.0 if (p == 1 and t == 0) else 0 for t, p in zip(predicted_class.T, true_class.T)])
    # print "No. of false positives:", fp
    fn = np.sum([1.0 if (p == 0 and t == 1) else 0 for t, p in zip(predicted_class.T, true_class.T)])
    # print "No. of false negatives:", fn
    if (tp + fp) == 0:
        return np.inf
    else:
        pre = tp / (tp + fp)
        rec = tp / (tp + fn)
        return 2 * pre * rec / (pre + rec)

In [64]:
# predicted y s assessed using f measure,
# and if the predicted y is > .5 the o/p is
# classified as y = 1, and 0 otherwisedef testing(x_test,y_test,wts_test):
def testing(x_test, y_test, wts_test):
    y_predicted = hyp_fn(wts_test, x_test)
    for i in range(80):
        op = y_predicted[0, i]
        if op > 0.5:
            y_predicted[0, i] = 1
        else:
            y_predicted[0, i] = 0
    # print "Truth value compared to the predicted class and the groud truth is :"
    f_meas = evaluatePridictedWithTrue(y_predicted, y_test)
    return f_meas

In [65]:
if __name__ == '__main__':
    # processing the data from files and relevant conversions
    file='csv_result-chronic_kidney_disease_full (1).csv'
    outfile = file

    data_column_wise = replaceMissingData(outfile)  # outfile must be csv
    n, no_samples = np.shape(data_column_wise)
    m1 = int(.1 * no_samples)
    m = int(.8 * no_samples)
    data_column_wise = normalizeData(data_column_wise, n - 1)  # uncomment this line for standardization
    train_x = np.copy(data_column_wise[:n - 1, m1:m + m1])  # data for training - 80%
    train_y = np.copy(data_column_wise[n - 1:, m1:m + m1])
    lambda_range = np.arange(-2.0, 4.2, .2)
    f_m = []
    for lambda_value in lambda_range:
        # print "lambda is =", lambda_value
        if lambda_value < -0.4 and lambda_value < 0:
            alpha = .002  # .01works for (1.0 or .9 ep)
            epsilon_tolerance = .19  # 1.6for standardization/normalization
        else:
            alpha = .002
            epsilon_tolerance = 21
        
        wts = findWeights(train_x, m, n - 1, lambda_value, epsilon_tolerance, alpha)
        
        test_x = np.copy(data_column_wise[:n - 1, :m1])  # data for testing - 20%
        test_x = np.concatenate((test_x, np.copy(data_column_wise[:n - 1, m1 + m:])), axis=1)
        y = np.copy(data_column_wise[n - 1:, :m1])  # true class
        y = np.concatenate((y, np.copy(data_column_wise[n - 1:, m1 + m:])), axis=1)
        temp_f_m = testing(test_x, y, wts)
        f_m.append(temp_f_m)
    
    # plotting f-measure vs lambda
    plt.figure("f-measure vs lambda")
    plt.title("f-measure vs lambda; alpha = .01")
    plt.plot(lambda_range, f_m, 'ro')
    plt.xlabel("lambda = -2.0 to 4.0 in steps of 0.2")
    plt.ylabel("f-measure")
    plt.ylim((0, 1))
    plt.show()

<ipython-input-46-ea25f1aab568>:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  columns = columns.astype(np.float)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()